# Реализовать суммаризацию текста

## Импорты

In [1]:
import numpy as np
import pandas as pd

import torch
from torch import LongTensor

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

## Настройки

In [2]:
DATA_PATH = 'IlyaGusev/gazeta'
MODEL = 'IlyaGusev/rugpt3medium_sum_gazeta'

## Данные

In [3]:
data = load_dataset('IlyaGusev/gazeta')

No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (C:/Users/avpat/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/e2d171980aa248bc22e0af4f8485ad69071fc8e5f3d54a253c71eb434f6694bd)


  0%|          | 0/3 [00:00<?, ?it/s]

## Модель

In [4]:
class Summarizer:
    def __init__(self, model, max_length=100, device='cpu'):
        self._tokenizer = AutoTokenizer.from_pretrained(model)
        self._model = AutoModelForCausalLM.from_pretrained(model)
        self._max_length = max_length
        self.to(device)

    def summarize(self, text, max_length=None):
        max_length = max_length if max_length else self._max_length
        tokens = self._tokenizer(text,
                                 add_special_tokens=False,
                                 padding=False)["input_ids"]

        tokens += [self._tokenizer.sep_token_id]
        tokens = LongTensor([tokens]).to(self._device)

        output_ids = self._model.generate(input_ids=tokens,
                                          max_length=tokens.shape[1]+1 + max_length,
                                          no_repeat_ngram_size=2,
                                          num_beams=4)

        summary = self._tokenizer.decode(output_ids[0], skip_special_tokens=False)
        summary = summary.split(self._tokenizer.sep_token)[1]
        summary = summary.split(self._tokenizer.eos_token)[0]
        return summary
    
    def to(self, device):
        self._model = self._model.to(device)
        self._device = device

summarizer = Summarizer(MODEL, max_length=70, device='cpu')

In [5]:
train = pd.DataFrame(data['train'])
train.head()

,text,summary,title,date,url
0,Сегодня транспортный налог начисляется в завис...,С 2011 года правительство отменяет самый раздр...,Налог в бак,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...
1,Словосочетание «музыкальный кинофестиваль» уже...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...
2,После более чем 12-часовых консультаций Совет ...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...
3,"Жертвами урагана «Агата», обрушившегося на Цен...",Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...
4,Решение ограничить рост тарифов естественных м...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...


In [6]:
i = 0
print('text:', train['text'][i])
print('\nDataset summary:', train['summary'][i])
print('\nSummary:', summarizer.summarize(train['text'][i]))

text: Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодня составляет

In [7]:
i = 1
print('text:', train['text'][i])
print('\nDataset summary:', train['summary'][i])
print('\nSummary:', summarizer.summarize(train['text'][i]))

text: Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий в этот четверг Beat Film Festival – прямой наследник Music.doc. На этот раз к услугам киномеломанов самая авторитетная арт-хаусная площадка города, а не тесный клуб или помпезный «Иллюзион», а вместо собранной по сусекам программы – фильмы с не более чем двухлетним сроком давности. Картины «Bassweight» и «Favellla on Blast» рассказывают (соответственно) о

In [8]:
i = 2
print('text:', train['text'][i])
print('\nDataset summary:', train['summary'][i])
print('\nSummary:', summarizer.summarize(train['text'][i]))

text: После более чем 12-часовых консультаций Совет Безопасности ООН согласовал заявление председателя по израильскому штурму «Флотилии свободы». Совбез осудил захват флотилии, требует немедленно отпустить суда и призывает к проведению международного расследования инцидента, сообщает BBC со ссылкой на дипломатические источники. Экстренное заседание СБ ООН было созвано по инициативе Турции и Ливана после того, как израильские военные перехватили шесть судов шедшего в Газу гуманитарного конвоя. Согласно сообщению минобороны Израиля, при захвате погибли девять человек, ранены еще 33 человека. По информации посольства Израиля в Москве, в операции пострадали 10 израильских военных. В ходе открытого обсуждения Турция выступила с резкой критикой действий Израиля. «Попросту говоря, это равносильно бандитизму и пиратству. Это убийство, осуществленное руками государства», — заявил в ходе заседания турецкий министр иностранных дел Ахмет Давутоглу. «Эта флотилия была чем угодно, только не гуманита